In [82]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
 
# import requests
 
# # getting text document from Internet
# text = requests.get('http://rare-technologies.com/the_matrix_synopsis.txt').text
 
 
# getting text document from file
fname="TextSummary/2170_001.txt"
with open(fname, 'r') as myfile:
      text=myfile.read()
     
     
# #getting text document from web, below function based from 3
# from bs4 import BeautifulSoup
# from urllib.request import urlopen
 
# def get_only_text(url):
#  """ 
#   return the title and the text of the article
#   at the specified url
#  """
#  page = urlopen(url)
#  soup = BeautifulSoup(page, "lxml")
#  text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
#  return soup.title.text, text    
 
  
print ('Summary:')
print (summarize(text, ratio=0.01))
 
print ('\nKeywords:')
print (keywords(text, ratio=0.01))
 
# url="https://en.wikipedia.org/wiki/Deep_learning"
# text = get_only_text(url)
 
# print ('Summary:')   
# print (summarize(str(text), ratio=0.01))
 
# print ('\nKeywords:')
 
# # higher ratio => more keywords
# print (keywords(str(text), ratio=0.01))

Summary:
Delayed inside information  Article 17(4) of MAR   
Where the issuer is also a credit or financial institution, Article 17(5) of MAR allows 
disclosure  of  inside  information  under  Article  17(4)  of  MAR  or  notifying  to  the 
inside  information  in  accordance  with  Article  17(4)  of  MAR  and,  due  to 
According  to  Article  17(1)  of  MAR,  an  issuer  has  to  inform  the  public  as  soon  as 
Where an issuer has delayed the disclosure of inside information according to Article 
Delayed disclosure of inside information under Article 17(5) of MAR: assessment of the 
Point a) of Article 17(5) of MAR: disclosure of inside information entails a risk 
Delayed disclosure of inside information under Article 17(5) of MAR: notification of the 
Delayed  disclosure  of inside information  under  Article  17(5)  of MAR:  NCA’s  denial  of 
financial instrument or an issuer would be considered as investment recommendation 
financial instrument would fall under Article 2(1)

text


In [17]:
#note FrequencySummarizer is need to be copied from
# https://glowingpython.blogspot.com/2014/09/text-summarization-with-nltk.html
# and saved as FrequencySummarizer.py in the same folder that this
# script
#from FrequencySummarizer import FrequencySummarizer
 
 
from bs4 import BeautifulSoup
from urllib.request import urlopen
 
 
# def get_only_text(url):
#  """ 
#   return the title and the text of the article
#   at the specified url
#  """
  
#  page = urlopen(url)
#  soup = BeautifulSoup(page)
#  text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
  
#  print ("=====================")
#  print (text)
#  print ("=====================")
 
#  return soup.title.text, text    
 
     
# url="https://en.wikipedia.org/wiki/Deep_learning"
# text = get_only_text(url)    
 
fs = FrequencySummarizer()
s = fs.summarize(str(text), 5)
print (s)



['17(5):  \n \n \n \n01/10/2018 \n(new)   \n01/10/2018 \n(new)   \nNCA’s denial of consent  \n \nPrevention   6.1 \nand detection   \nPersons professionally  \narranging or executing  \nof market  \nabuse   \n \n \n \ntransactions \n \nArticle 16(2) of MAR   \n1/09/2017 \nManagers’ \n7.1 \nExchange rate \n \nArticle 19 of MAR \n \n26/10/2016 \n \n7.2 \ntransactions \nManagers’ \nTransactions   \nManagers’ \ntransactions \nManagers’ \ntransactions    \nManagers’ \n7.3 \n \n7.4 \n7.5 \n \nTiming of the closed    \nperiod \nThreshold calculation  \n \n \nPrice of gifts, donation  \nand inheritance  \n \nNotification of shares   \nArticle 19(11) of MAR  \n13/07/2016 \nArticle 19(1) and(8) of MAR  20/12/2016 \nArticle 10(2)(k) of  \nRegulation 2016/522 \nArticle 19(1) of MAR   \n \n20/12/2016 \n20/12/2016 \nTransactions   \nreceived as part of  \n \nArticle 10(2)(i) of \n \n  \nManagers’ \n \n7.6 \nremuneration package  \nPrice of options granted \nRegulation 2016/522 \nArticle 10(2)(k) of 

In [16]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest

class FrequencySummarizer:
  def __init__(self, min_cut=0.1, max_cut=0.9):
    """
     Initilize the text summarizer.
     Words that have a frequency term lower than min_cut 
     or higer than max_cut will be ignored.
    """
    self._min_cut = min_cut
    self._max_cut = max_cut 
    self._stopwords = set(stopwords.words('english') + list(punctuation))

  def _compute_frequencies(self, word_sent):
    """ 
      Compute the frequency of each of word.
      Input: 
       word_sent, a list of sentences already tokenized.
      Output: 
       freq, a dictionary where freq[w] is the frequency of w.
    """
    freq = defaultdict(int)
    for s in word_sent:
      for word in s:
        if word not in self._stopwords:
          freq[word] += 1
    # frequencies normalization and fitering
    m = float(max(freq.values()))
    #for w in freq.keys():
    for w in list(freq):
      freq[w] = freq[w]/m
      if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
        del freq[w]
    return freq

  def summarize(self, text, n):
    """
      Return a list of n sentences 
      which represent the summary of text.
    """
    sents = sent_tokenize(text)
    assert n <= len(sents)
    word_sent = [word_tokenize(s.lower()) for s in sents]
    self._freq = self._compute_frequencies(word_sent)
    ranking = defaultdict(int)
    for i,sent in enumerate(word_sent):
      for w in sent:
        if w in self._freq:
          ranking[i] += self._freq[w]
    sents_idx = self._rank(ranking, n)    
    return [sents[j] for j in sents_idx]

  def _rank(self, ranking, n):
    """ return the first n sentences with highest ranking """
    return nlargest(n, ranking, key=ranking.get)


In [48]:
text = 'Dit is zo maar een verhaalCorpus means a collection of text. It could be data sets of poems by a certain poet, bodies of work by a certain author, etc. In this case, we are going to use a data set of pre-determined stop words.Tokenizers. Basically, it divides a text into a series of tokens. There are three main tokenizers - word, sentence, and regex tokenizer. For this specific project, we will only use the word and sentence tokenizer.'

In [78]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stopWords = set(stopwords.words("english"))
words = word_tokenize(text)
freqTable = dict()
for word in words:
    word = word.lower()
    if word in stopWords:
        continue
    if word in freqTable:
        freqTable[word] += 1
    else:
        freqTable[word] = 1
sentences = sent_tokenize(text)
sentenceValue = dict()
for sentence in sentences:
     for index, wordValue in enumerate(freqTable, start=1):
        if wordValue in sentence.lower():  # index[0] return word
               if sentence in sentenceValue:
                    # index return value of occurence of that word
                    sentenceValue[sentence] += index
                    # sentenceValue.update({sentence: index})
                    # print(sentenceValue)
        else:
                   # sentenceValue[sentence] = wordValue
                    sentenceValue[sentence] = index
                    print(sentenceValue)
sumValues = 0
for sentence in sentenceValue:
    sumValues += sentenceValue[sentence]

# # Average value of a sentence from original text
# average = int(sumValues / len(sentenceValue))
# summary = ''
# for sentence in sentences:
#         if sentence[:12] in sentenceValue and sentenceValue[sentence[:12] > (1.5 * average):
#             summary += " " + sentence

{'Dit is zo maar een verhaalCorpus means a collection of text.': 10}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 11}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 12}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 13}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 14}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 15}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 16}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 17}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 18}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 19}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 20}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 21}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 22}
{'Dit is zo maar een verhaalCorpus means a collection of text.': 23}
{'Dit is zo maar een verhaalCorpus

In [74]:
summary

''

In [75]:
sentenceValue

{'Basically, it divides a text into a series of tokens.': 41,
 'Dit is zo maar een verhaalCorpus means a collection of text.': 41,
 'For this specific project, we will only use the word and sentence tokenizer.': 158,
 'In this case, we are going to use a data set of pre-determined stop words.Tokenizers.': 41,
 'It could be data sets of poems by a certain poet, bodies of work by a certain author, etc.': 41,
 'There are three main tokenizers - word, sentence, and regex tokenizer.': 41}

In [57]:
freqTable.values()

dict_values([1, 1, 1, 1, 1, 1, 1, 2, 6, 1, 2, 1, 1, 2, 1, 7, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1])

In [52]:
pprint(freqTable)

{',': 7,
 '-': 1,
 '.': 6,
 'author': 1,
 'basically': 1,
 'bodies': 1,
 'case': 1,
 'certain': 2,
 'collection': 1,
 'could': 1,
 'data': 2,
 'dit': 1,
 'divides': 1,
 'een': 1,
 'etc': 1,
 'going': 1,
 'maar': 1,
 'main': 1,
 'means': 1,
 'poems': 1,
 'poet': 1,
 'pre-determined': 1,
 'project': 1,
 'regex': 1,
 'sentence': 2,
 'series': 1,
 'set': 1,
 'sets': 1,
 'specific': 1,
 'stop': 1,
 'text': 2,
 'three': 1,
 'tokenizer': 2,
 'tokenizers': 1,
 'tokens': 1,
 'use': 2,
 'verhaalcorpus': 1,
 'word': 2,
 'words.tokenizers': 1,
 'work': 1,
 'zo': 1}


In [79]:
# Simple Summarizer
# Copyright (C) 2010-2012 Tristan Havelick
# Author: Tristan Havelick <tristan@havelick.com>
# URL: <https://github.com/thavelick/summarize/>
# For license information, see LICENSE.TXT

"""
A summarizer based on the algorithm found in Classifier4J by Nick Lothan.
In order to summarize a document this algorithm first determines the
frequencies of the words in the document.  It then splits the document
into a series of sentences.  Then it creates a summary by including the
first sentence that includes each of the most frequent words.  Finally
summary's sentences are reordered to reflect that of those in the original
document.
"""

##//////////////////////////////////////////////////////
##  Simple Summarizer
##//////////////////////////////////////////////////////

from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk.data

class SimpleSummarizer:

	def reorder_sentences( self, output_sentences, input ):
		output_sentences.sort( lambda s1, s2:
			input.find(s1) - input.find(s2) )
		return output_sentences
	
	def get_summarized(self, input, num_sentences ):
		# TODO: allow the caller to specify the tokenizer they want
		# TODO: allow the user to specify the sentence tokenizer they want
		
		tokenizer = RegexpTokenizer('\w+')
		
		# get the frequency of each word in the input
		base_words = [word.lower()
			for word in tokenizer.tokenize(input)]
		words = [word for word in base_words if word not in stopwords.words()]
		word_frequencies = FreqDist(words)
		
		# now create a set of the most frequent words
		most_frequent_words = [pair[0] for pair in
			word_frequencies.items()[:100]]
		
		# break the input up into sentences.  working_sentences is used
		# for the analysis, but actual_sentences is used in the results
		# so capitalization will be correct.
		
		sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
		actual_sentences = sent_detector.tokenize(input)
		working_sentences = [sentence.lower()
			for sentence in actual_sentences]

		# iterate over the most frequent words, and add the first sentence
		# that inclues each word to the result.
		output_sentences = []

		for word in most_frequent_words:
			for i in range(0, len(working_sentences)):
				if (word in working_sentences[i]
				  and actual_sentences[i] not in output_sentences):
					output_sentences.append(actual_sentences[i])
					break
				if len(output_sentences) >= num_sentences: break
			if len(output_sentences) >= num_sentences: break
			
		# sort the output sentences back to their original order
		return self.reorder_sentences(output_sentences, input)
	
	def summarize(self, input, num_sentences):
		return " ".join(self.get_summarized(input, num_sentences))

In [81]:
ss = SimpleSummarizer()
input = "NLTK is a python library for working human-written text. Summarize is a package that uses NLTK to create summaries."
ss.summarize(input, 1)

TypeError: 'dict_items' object is not subscriptable

In [86]:
import sumy

In [109]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
import sumy
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers import text_rank
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer   #found this is the best as 
# it is picking from beginning also while other skip
 
 
LANGUAGE = "english"
SENTENCES_COUNT = 10
stemmer = Stemmer(LANGUAGE)
 
if __name__ == "__main__":
    
#     url="https://en.wikipedia.org/wiki/Deep_learning"
   
#     parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    # or for plain text files
    parser = PlaintextParser.from_file("TextSummary/2170_001.txt", Tokenizer(LANGUAGE))
    
#     print ("--Lexrank--")    
#     summarizer = ()
#     summarizer = LexRankSummarizer(Stemmer(LANGUAGE))
#     summarizer.stop_words = get_stop_words(LANGUAGE)
#     for sentence in summarizer(parser.document, SENTENCES_COUNT):
#         print(sentence)
        
    print ("--sumbasic--")    
    summarizer = SumBasicSummarizer(stemmer)
#     summarizer = sum_basic(Stemmer(LANGUAGE))
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
        
#     print ("--LsaSummarizer--")    
#     summarizer = LsaSummarizer()
#     summarizer = LsaSummarizer(Stemmer(LANGUAGE))
#     summarizer.stop_words = get_stop_words(LANGUAGE)
#     for sentence in summarizer(parser.document, SENTENCES_COUNT):
#         print(sentence)
         
#     print ("--LuhnSummarizer--")     
#     summarizer = LuhnSummarizer() 
#     summarizer = LsaSummarizer(Stemmer(LANGUAGE))
#     summarizer.stop_words = ("I", "am", "the", "you", "are", "me", "is", "than", "that", "this",)
#     for sentence in summarizer(parser.document, SENTENCES_COUNT):
#         print(sentence)
         
#     print ("--EdmundsonSummarizer--")     
#     summarizer = EdmundsonSummarizer() 
#     words = ("deep", "learning", "neural" )
#     summarizer.bonus_words = words
     
#     words = ("another", "and", "some", "next",)
#     summarizer.stigma_words = words
    
     
#     words = ("another", "and", "some", "next",)
#     summarizer.null_words = words
#     for sentence in summarizer(parser.document, SENTENCES_COUNT):
#         print(sentence)   

#words = [word for word in tokens if word.isalpha()]

--sumbasic--
Article 17 of MAR
30/05/2017 23/03/2018 of Inside information
transactions
Market soundings 9.1
Insider lists
According  to  Article  17(1)  of  MAR,  an  issuer  has  to  inform  the  public  as  soon  as possible of inside information that directly concerns that issuer.
In the period before MiFIR becomes applicable, the price to use will be:
financial
Q8.10  How  does  Commission  Delegated  Regulation  (EU)  2016/958  apply  when  the subject  of  the  recommendation  relates  to  several  financial  instruments independently?
Updated: 14 December 2017


In [94]:
len(get_stop_words(LANGUAGE))

580